In [25]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import numpy as np 
import os
import cv2
import pandas as pd 
from PIL import Image
from pathlib import Path
import seaborn as sb
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
for dirname, _, filenames in os.walk('/Users/mofeoluwajide-jegede/Documents/archive'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/mofeoluwajide-jegede/Documents/archive/food_c101_n1000_r384x384x3.h5
/Users/mofeoluwajide-jegede/Documents/archive/food_test_c101_n1000_r32x32x3.h5
/Users/mofeoluwajide-jegede/Documents/archive/food_test_c101_n1000_r128x128x1.h5
/Users/mofeoluwajide-jegede/Documents/archive/.DS_Store
/Users/mofeoluwajide-jegede/Documents/archive/food_c101_n10099_r32x32x1.h5
/Users/mofeoluwajide-jegede/Documents/archive/food_test_c101_n1000_r64x64x3.h5
/Users/mofeoluwajide-jegede/Documents/archive/food_c101_n10099_r64x64x1.h5
/Users/mofeoluwajide-jegede/Documents/archive/food_test_c101_n1000_r64x64x1.h5
/Users/mofeoluwajide-jegede/Documents/archive/food_c101_n10099_r64x64x3.h5
/Users/mofeoluwajide-jegede/Documents/archive/food_test_c101_n1000_r32x32x1.h5
/Users/mofeoluwajide-jegede/Documents/archive/food_c101_n10099_r32x32x3.h5
/Users/mofeoluwajide-jegede/Documents/archive/food_test_c101_n1000_r128x128x3.h5
/Users/mofeoluwajide-jegede/Documents/archive/images/.DS_Store
/Users/mofeoluwajide-jegede

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
image_dir = Path('/Users/mofeoluwajide-jegede/Documents/archive')

In [28]:
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Image_filepath').astype(str)
labels = pd.Series(labels, name='Label')

images = pd.concat([filepaths, labels], axis=1)

category_samples = []
for category in images['Label'].unique():
    category_slice = images.query("Label == @category")
    category_samples.append(category_slice.sample(100, random_state=1))
food_image_df = pd.concat(category_samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)


In [29]:
food_image_df

,Filepath,Label
0,/Users/mofeoluwajide-jegede/Documents/archive/...,pork_chop
1,/Users/mofeoluwajide-jegede/Documents/archive/...,pho
2,/Users/mofeoluwajide-jegede/Documents/archive/...,donuts
3,/Users/mofeoluwajide-jegede/Documents/archive/...,caesar_salad
4,/Users/mofeoluwajide-jegede/Documents/archive/...,sashimi
...,...,...
10095,/Users/mofeoluwajide-jegede/Documents/archive/...,greek_salad
10096,/Users/mofeoluwajide-jegede/Documents/archive/...,pancakes
10097,/Users/mofeoluwajide-jegede/Documents/archive/...,tuna_tartare
10098,/Users/mofeoluwajide-jegede/Documents/archive/...,fish_and_chips


In [30]:
food_image_df.Label.value_counts()

pork_chop        100
bread_pudding    100
club_sandwich    100
french_fries     100
beef_tartare     100
                ... 
creme_brulee     100
hummus           100
clam_chowder     100
caprese_salad    100
foie_gras        100
Name: Label, Length: 101, dtype: int64

In [45]:
train_df, test_df = train_test_split(images, train_size=0.7, shuffle=True, random_state=1)

In [46]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [47]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

Found 56560 validated image filenames belonging to 101 classes.


In [48]:

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 14140 validated image filenames belonging to 101 classes.
Found 30300 validated image filenames belonging to 101 classes.


In [49]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

In [50]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(101, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [51]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
1768/1768 [==============================] - 1709s 965ms/step - loss: 2.3326 - accuracy: 0.4182 - val_loss: 1.9319 - val_accuracy: 0.5075
Epoch 2/100
1768/1768 [==============================] - 1547s 875ms/step - loss: 1.7461 - accuracy: 0.5443 - val_loss: 1.8272 - val_accuracy: 0.5332
Epoch 3/100
1768/1768 [==============================] - 2713s 2s/step - loss: 1.5584 - accuracy: 0.5860 - val_loss: 1.8252 - val_accuracy: 0.5426
Epoch 4/100
1768/1768 [==============================] - 1706s 965ms/step - loss: 1.4323 - accuracy: 0.6158 - val_loss: 1.8347 - val_accuracy: 0.5426
Epoch 5/100
1768/1768 [==============================] - 2040s 1s/step - loss: 1.3159 - accuracy: 0.6426 - val_loss: 1.8991 - val_accuracy: 0.5436
Epoch 6/100
1768/1768 [==============================] - 1838s 1s/step - loss: 1.2249 - accuracy: 0.6661 - val_loss: 1.9179 - val_accuracy: 0.5418


In [52]:
results = model.evaluate(test_images, verbose=0)
print("Test Accuracy: {:.2f}%".format(results[1] * 100))


Test Accuracy: 53.17%


In [53]:
predictions = np.argmax(model.predict(test_images), axis=1)

cm = confusion_matrix(test_images.labels, predictions)
clr = classification_report(test_images.labels, predictions, target_names=test_images.class_indices, zero_division=0)

947/947 [==============================] - 896s 945ms/step


In [54]:
print("Classification Report:\n----------------------\n", clr)


Classification Report:
----------------------
                          precision    recall  f1-score   support

              apple_pie       0.36      0.14      0.20       308
         baby_back_ribs       0.70      0.48      0.57       297
                baklava       0.46      0.66      0.54       289
         beef_carpaccio       0.52      0.66      0.58       294
           beef_tartare       0.49      0.37      0.42       297
             beet_salad       0.41      0.45      0.43       289
               beignets       0.74      0.67      0.70       302
               bibimbap       0.82      0.67      0.74       303
          bread_pudding       0.17      0.60      0.26       291
      breakfast_burrito       0.57      0.37      0.45       304
             bruschetta       0.39      0.44      0.41       278
           caesar_salad       0.64      0.55      0.59       297
                cannoli       0.42      0.61      0.50       287
          caprese_salad       0.45      0.

In [31]:
train_df, test_df = train_test_split(food_image_df, train_size=0.7, shuffle=True, random_state=1)

In [32]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [33]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

Found 5656 validated image filenames belonging to 101 classes.


In [34]:
val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)


Found 1414 validated image filenames belonging to 101 classes.


In [35]:

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 3030 validated image filenames belonging to 101 classes.


In [36]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

2023-03-14 20:18:24.130214: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


9406464/9406464 [==============================] - 1s 0us/step


In [37]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(101, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [39]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
177/177 [==============================] - 163s 910ms/step - loss: 3.8048 - accuracy: 0.1457 - val_loss: 2.9194 - val_accuracy: 0.3034
Epoch 2/100
177/177 [==============================] - 152s 860ms/step - loss: 2.3714 - accuracy: 0.4035 - val_loss: 2.5194 - val_accuracy: 0.3600
Epoch 3/100
177/177 [==============================] - 170s 962ms/step - loss: 1.7971 - accuracy: 0.5362 - val_loss: 2.4231 - val_accuracy: 0.3946
Epoch 4/100
177/177 [==============================] - 187s 1s/step - loss: 1.4285 - accuracy: 0.6246 - val_loss: 2.4912 - val_accuracy: 0.3904
Epoch 5/100
177/177 [==============================] - 204s 1s/step - loss: 1.1363 - accuracy: 0.6922 - val_loss: 2.4365 - val_accuracy: 0.4081
Epoch 6/100
177/177 [==============================] - 213s 1s/step - loss: 0.8851 - accuracy: 0.7617 - val_loss: 2.4877 - val_accuracy: 0.4137


In [40]:
results = model.evaluate(test_images, verbose=0)
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

Test Accuracy: 39.60%


In [41]:
predictions = np.argmax(model.predict(test_images), axis=1)

cm = confusion_matrix(test_images.labels, predictions)
clr = classification_report(test_images.labels, predictions, target_names=test_images.class_indices, zero_division=0)

95/95 [==============================] - 70s 725ms/step


In [42]:
print("Classification Report:\n----------------------\n", clr)


Classification Report:
----------------------
                          precision    recall  f1-score   support

              apple_pie       0.00      0.00      0.00        44
         baby_back_ribs       0.27      0.57      0.36        28
                baklava       0.23      0.44      0.30        32
         beef_carpaccio       0.25      0.53      0.34        32
           beef_tartare       0.21      0.46      0.29        26
             beet_salad       0.16      0.27      0.20        30
               beignets       0.52      0.70      0.60        33
               bibimbap       0.32      0.83      0.46        24
          bread_pudding       0.41      0.38      0.39        34
      breakfast_burrito       0.44      0.17      0.24        24
             bruschetta       1.00      0.10      0.19        29
           caesar_salad       0.85      0.37      0.51        30
                cannoli       0.25      0.63      0.36        27
          caprese_salad       0.83      0.